# SNOTEL depth data processor

This notebook assists with processing daily snow data in inches (SNOTEL format)
into hourly snow depth in cm (SAM snow model format)

In [2]:
# Import required packages
import numpy as np
import pandas as pd
import csv

Set the file path for the downloaded data

SNOTEL data is formatted by water year: 2003 means October 2002 to September 2003
NSRDB data is calendar year.
Calendar year end should be calender year start + 1

In [1]:
filename = "Crosho.csv"
calendar_year_start = "2003"
calendar_year_end = "2004" # TODO: consider automating this with string conversion, either here or in pandas
out_filename = "crosho_2003.csv"

Run the cell below to import the data as a dataframe and get an array of 365 days

In [3]:
df = pd.read_csv(filename)

last_3_mo = 31+30+31 # Days in Oct to Sec 
first_2_mo = 31+28 # Jan and Feb - skips leap days
next_9_mo = last_3_mo + first_2_mo + 1 # Start point 9 months before end of calendar year

jan_feb = df[calendar_year_start].loc[(last_3_mo):(last_3_mo + first_2_mo - 1)].values
mar_sept = df[calendar_year_start ].loc[next_9_mo:].values
oct_dec = df[calendar_year_end].loc[:(last_3_mo - 1)].values

ay_data = np.concatenate([jan_feb, mar_sept, oct_dec], axis=0)

Convert from hourly data to daily data and inches to cm. Ends with 8760 array

In [3]:
in_to_cm = 2.54
timeseries_data = []
for d in ay_data:
    for i in range(0, 24):
        timeseries_data.append(d * in_to_cm)

# print(len(timeseries_data)) # Uncomment for debugging

8760


Write array to file

In [4]:

file = open(out_filename, 'w+', newline ='')

with file:
    write = csv.writer(file)
    write.writerows(map(lambda x: [x], timeseries_data))

In [52]:
# Debugging and data applicability check: sum of hourly snow depths
sum(timeseries_data)

492191.04000000097